In [2]:
import pandas as pd

# ---------- SETTINGS ----------
# Zet hier de bestandsnamen van je CSV's
MENTOREN_FILE = "../data/mentoren.csv"
STUDENTEN_FILE = "../data/studenten.csv"
OUTPUT_FILE = "../output/matches.csv"

# ---------- DATA INLADEN ----------
mentoren_df = pd.read_csv(MENTOREN_FILE)
studenten_df = pd.read_csv(STUDENTEN_FILE)

# Zorg dat onderwerpen correct als lijst worden gelezen
mentoren_df["Onderwerpen"] = mentoren_df["Onderwerpen"].apply(
    lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x]
)

# Opleidingsniveaus ordenen
level_order = {"Associate": 0, "Bachelor": 1, "Master": 2, "PhD": 3}

# ---------- MATCHING FUNCTIE ----------
def find_best_mentor(student, mentoren_df):
    student_level = level_order.get(student["Opleidingsniveau"], -1)
    student_subject = student["Onderwerp"]

    best_match = None
    best_score = -1

    for _, mentor in mentoren_df.iterrows():
        mentor_level = level_order.get(mentor["Opleidingsniveau"], -1)

        # Mentor moet hoger niveau hebben
        if mentor_level <= student_level:
            continue

        # Onderwerp match
        subject_match = 1 if student_subject in mentor["Onderwerpen"] else 0

        # Score = niveauverschil + extra punten voor onderwerp
        score = (mentor_level - student_level) + (subject_match * 2)

        if score > best_score:
            best_score = score
            best_match = mentor

    if best_match is not None:
        return {
            "Student": f"{student['Voornaam']} {student['Achternaam']}",
            "Student_niveau": student["Opleidingsniveau"],
            "Onderwerp": student_subject,
            "Mentor": f"{best_match['Voornaam']} {best_match['Achternaam']}",
            "Mentor_niveau": best_match["Opleidingsniveau"],
            "Mentor_onderwerpen": best_match["Onderwerpen"],
            "Match_score": best_score,
        }
    else:
        return {
            "Student": f"{student['Voornaam']} {student['Achternaam']}",
            "Student_niveau": student["Opleidingsniveau"],
            "Onderwerp": student_subject,
            "Mentor": None,
            "Mentor_niveau": None,
            "Mentor_onderwerpen": None,
            "Match_score": 0,
        }

# ---------- MATCHES MAKEN ----------
matches = [find_best_mentor(student, mentoren_df) for _, student in studenten_df.iterrows()]
matches_df = pd.DataFrame(matches)

# ---------- RESULTAAT OPSLAAN ----------
matches_df.to_csv(OUTPUT_FILE, index=False)
print(f"Matching voltooid! Resultaat opgeslagen in {OUTPUT_FILE}")

Matching voltooid! Resultaat opgeslagen in ../output/matches.csv


In [6]:
import random
import pandas as pd


class Student:
    def __init__(self, naam: str, niveau: str, onderwerp: str):
        self.naam = naam
        self.niveau = niveau
        self.onderwerp = onderwerp

    def __repr__(self):
        return f"Student(naam={self.naam}, niveau={self.niveau}, onderwerp={self.onderwerp})"

class Mentor:
    def __init__(self, naam: str, niveau: str, onderwerpen: list[str], max_studenten: int):
        self.naam = naam
        self.niveau = niveau
        self.onderwerpen = onderwerpen
        self.max_studenten = max_studenten

    def __repr__(self):
        return f"Mentor(naam={self.naam}, niveau={self.niveau}, onderwerpen={self.onderwerpen}, max_studenten={self.max_studenten})"
    

def vul_studenten(studenten_lijst, csv_path, n):
    df = pd.read_csv(csv_path)
    rows = df.sample(n=min(n, len(df)))
    for _, row in rows.iterrows():
        naam = f"{row['Voornaam']} {row['Achternaam']}" if 'Voornaam' in row and 'Achternaam' in row else row.get('Naam', '')
        studenten_lijst.append(Student(naam, row['Opleidingsniveau'], row['Onderwerp']))


def vul_mentoren(mentoren_lijst, csv_path, n):
    df = pd.read_csv(csv_path)
    # Zorg dat onderwerpen als lijst worden gelezen
    df["Onderwerpen"] = df["Onderwerpen"].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x])
    rows = df.sample(n=min(n, len(df)))
    for _, row in rows.iterrows():
        naam = f"{row['Voornaam']} {row['Achternaam']}" if 'Voornaam' in row and 'Achternaam' in row else row.get('Naam', '')
        max_studenten = random.randint(1, 5)
        mentoren_lijst.append(Mentor(naam, row['Opleidingsniveau'], row['Onderwerpen'], max_studenten))


studenten = []
mentoren = []

vul_studenten(studenten, "../data/studenten.csv", 90)
vul_mentoren(mentoren, "../data/mentoren.csv", 30)

print(studenten)
print(mentoren)




[Student(naam=Niek Tillmanno, niveau=Associate, onderwerp=Software Architecture), Student(naam=Arie Charon, niveau=Bachelor, onderwerp=Business Process Analytics), Student(naam=Ayoub van der Velden, niveau=Bachelor, onderwerp=Ethical Hacking), Student(naam=Aimée Dachgelt, niveau=Associate, onderwerp=Data Visualisation), Student(naam=Megan Kleijse, niveau=Bachelor, onderwerp=Artificial Intelligence), Student(naam=Arthur Jacobi, niveau=Associate, onderwerp=Creative Digital Innovation), Student(naam=Kate Demmendaal, niveau=Associate, onderwerp=Design Science Research), Student(naam=Dina van Maaren, niveau=Associate, onderwerp=Data Visualisation), Student(naam=Jonathan Verbeek, niveau=Bachelor, onderwerp=Data Visualisation), Student(naam=Noortje Kuijpers, niveau=Bachelor, onderwerp=Business Process Analytics), Student(naam=Tessa Molen, niveau=Associate, onderwerp=Creative Digital Innovation), Student(naam=Dion van den Corput, niveau=Bachelor, onderwerp=Data Visualisation), Student(naam=Mar

# Uninformed 
bfs (werkt maar alleen voor kleine aantallen)

In [9]:
from collections import deque
import copy

def bfs_student_mentor_matching(studenten, mentoren):
    mentor_slots = {i: mentoren[i].max_studenten for i in range(len(mentoren))}
    initial_state = (0, [], mentor_slots)
    queue = deque([initial_state])

    while queue:
        student_idx, assignment, slots = queue.popleft()
        if student_idx == len(studenten):
            return assignment

        student = studenten[student_idx]
        assigned = False
        for mentor_idx, mentor in enumerate(mentoren):
            if (
                slots[mentor_idx] > 0 
                and student.onderwerp in mentor.onderwerpen
                and level_order[mentor.niveau] > level_order[student.niveau]
            ):
                new_slots = copy.deepcopy(slots)
                new_slots[mentor_idx] -= 1
                queue.append((student_idx + 1, assignment + [mentor_idx], new_slots))
                assigned = True
        if not assigned:
            queue.append((student_idx + 1, assignment + [None], slots))
    return None  # No solution found

# Voorbeeld van gebruik:
result = bfs_student_mentor_matching(studenten, mentoren)
for idx, mentor_idx in enumerate(result):
    student = studenten[idx]
    mentor = mentoren[mentor_idx] if mentor_idx is not None else None
    print(f"{student.naam} -> {mentor.naam if mentor else 'GEEN MATCH'}")

KeyboardInterrupt: 

In [7]:
import copy

def dfs_student_mentor_matching(studenten, mentoren):
    mentor_slots = {i: mentoren[i].max_studenten for i in range(len(mentoren))}
    stack = [(0, [], mentor_slots)]

    while stack:
        student_idx, assignment, slots = stack.pop()
        if student_idx == len(studenten):
            return assignment

        student = studenten[student_idx]
        assigned = False
        for mentor_idx, mentor in enumerate(mentoren):
            if (
                slots[mentor_idx] > 0
                and student.onderwerp in mentor.onderwerpen
                and level_order[mentor.niveau] > level_order[student.niveau]
            ):
                new_slots = copy.deepcopy(slots)
                new_slots[mentor_idx] -= 1
                stack.append((student_idx + 1, assignment + [mentor_idx], new_slots))
                assigned = True
        if not assigned:
            stack.append((student_idx + 1, assignment + [None], slots))
    return None  # No solution found

# Voorbeeld van gebruik:
result = dfs_student_mentor_matching(studenten, mentoren)
for idx, mentor_idx in enumerate(result):
    student = studenten[idx]
    mentor = mentoren[mentor_idx] if mentor_idx is not None else None
    print(f"{student.naam} -> {mentor.naam if mentor else 'GEEN MATCH'}")

Niek Tillmanno -> Livia van Luyssel
Arie Charon -> Kevin Osterhoudt
Ayoub van der Velden -> Adriana Schelvis
Aimée Dachgelt -> Myrthe Rijks
Megan Kleijse -> Helena Pieters
Arthur Jacobi -> Kevin Osterhoudt
Kate Demmendaal -> Nikki van de Wiel
Dina van Maaren -> Myrthe Rijks
Jonathan Verbeek -> Myrthe Rijks
Noortje Kuijpers -> Luka Sanders
Tessa Molen -> Nikki van de Wiel
Dion van den Corput -> Myrthe Rijks
Mart Kuijpers -> Adriana Schelvis
Isis Verbeek -> Adriana Schelvis
Mason van der Horst -> Luka Sanders
Abel van Metz -> Alicia de Vries
Tyler Miltenburg -> Maarten Walsteijn
Nynke Verbeeck -> Ryan van Arkel
Esther Scholten -> Veerle Aertsz
Emir Jorrisen -> Helena Pieters
Tijn Doorhof -> Ryan van Arkel
Floor Merkx -> Helena Pieters
Roan Passchiers -> Alicia de Vries
Tijs Strijker -> Helena Pieters
Daan Moenen -> Ryan van Arkel
Mika Van Bragt -> Veerle Aertsz
Kyra Broeckx -> Chris van Landen
Bram Marceron -> Chris van Landen
Micha Claesdr -> Helena Pieters
Jelte Bourgondië, van -> Aali